In [1]:
!pip install selenium chromedriver_autoinstaller


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: C:\Users\HAEA_MI\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import chromedriver_autoinstaller
from selenium.webdriver.support.ui import Select

chromedriver_autoinstaller.install()
options = Options()
driver = webdriver.Chrome('./chromedriver', chrome_options=options)

C:\Users\HAEA_MI\AppData\Local\Temp\ipykernel_1956\3712292900.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver', chrome_options=options)
C:\Users\HAEA_MI\AppData\Local\Temp\ipykernel_1956\3712292900.py:8: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('./chromedriver', chrome_options=options)


In [6]:
from utils import * 
driver.get("https://mcols.autoever.com/secure/Dashboard.jspa")

ID = "10896665"
PW = "A12345678!"

enter_to_mcols(driver, ID, PW)
press_create_btn(driver)

In [7]:
import time
import json
def set_project(driver, project_option, issuetype_option) :

    project_field = driver.find_element(By.CLASS_NAME, 'issue-setup-fields').find_element(By.ID, 'project-field')
    project_field.click()
    project_field.send_keys(Keys.CONTROL + 'a' + Keys.NULL, '')
    project_field.send_keys(project_option)
    project_field.send_keys(Keys.ENTER)

    time.sleep(5)

    issuetype_field = driver.find_element(By.CLASS_NAME, 'issue-setup-fields').find_element(By.ID, 'issuetype-field')
    issuetype_field.click()
    issuetype_field.send_keys(Keys.CONTROL + 'a' + Keys.NULL, '')
    issuetype_field.send_keys(issuetype_option)
    issuetype_field.send_keys(Keys.ENTER)

    time.sleep(5)

def get_project_options(driver) :

    project_fields = driver.find_element(By.CLASS_NAME, 'issue-setup-fields')
    project_options = list(map(lambda x : x['label'], json.loads(project_fields.find_element(By.ID, 'project-options').get_attribute('data-suggestions'))[0]['items']))
    issuetype_options = list(map(lambda x : x['label'], json.loads(project_fields.find_element(By.ID, 'issuetype-options').get_attribute('data-suggestions'))[0]['items']))

    return {
        'projects' : project_options,
        'issuetypes' : issuetype_options
    }    

In [8]:
options = get_project_options(driver)

In [9]:
set_project(driver, options['projects'][0], options['issuetypes'][0])

In [63]:
def get_project_input_fields_json(driver, num_recommend_opton = -1) :
    
    tab = driver.find_element(By.ID, "tab-0")
    # field_items = tab.find_elements(By.CLASS_NAME, "field-group") + tab.find_elements(By.TAG_NAME, "fieldset")
    # field_selet_items = list(map(lambda x : {x.text.split('\n')[0]: x.find_elements(By.TAG_NAME, 'select')}, field_items))
    children = tab.find_elements(By.XPATH, "*")

    input_form = {}

    for child in children :

        # initialize inputs
        input_type = None
        xpath = None
        options = None
        required = False

        # check input element is required or not
        if  len(child.find_elements(By.CLASS_NAME, 'icon-required')) > 0 :
            required = True
        else :
            required = False

        child_class = child.get_attribute('class')
        if 'field-group' in child_class :

            i = child

            field_name = child.find_element(By.TAG_NAME, 'label').get_attribute("innerText").split('\n')[0]

            if 'aui-field-datepicker' in child_class :
                input_type = 'date'
                options = ""
                xpath = child

            elif 'frother-control-renderer' in child_class :
                input_type = 'rendering select'
                select_item_list = child.find_elements(By.TAG_NAME, 'select')
                option_list = select_item_list[0].find_elements(By.TAG_NAME, 'option')
                if len(option_list) > num_recommend_opton :
                    option_list = option_list[:num_recommend_opton]
                options = list(map(lambda x : x.get_attribute("innerText").strip(), option_list))
                xpath = child
            
            elif 'aui-field-cascadingselect' in child_class :
                select_item_list = child.find_elements(By.TAG_NAME, 'select')
                if child.get_attribute('style') != "display: none;" :
                    first_select = Select(select_item_list[0])
                    first_options = list(map(lambda x : x.text, first_select.options))
                    second_select = Select(select_item_list[1])
                    options = {}
                    for first_option in first_options :
                        first_select.select_by_visible_text(first_option)
                        second_options = list(map(lambda x : x.text, second_select.options))
                        options[first_option] = second_options
                else :                        
                    last_added_key = list(input_form.keys())[-1]
                    first_select = Select(input_form[last_added_key]['xpath'])
                    first_options = input_form[last_added_key]['options']
                    second_select = Select(select_item_list[0])
                    third_select = Select(select_item_list[1])
                    # options = "tbd"
                    options = {}
                    for first_option in first_options :
                        if first_option :
                            first_select.select_by_visible_text(first_option)
                            second_options = list(map(lambda x : x.text, second_select.options))
                            
                            options[first_option] = {}

                            for second_option in second_options :
                                if second_option :
                                    second_select.select_by_visible_text(second_option)
                                    third_options = list(map(lambda x : x.text, third_select.options))
                                    
                                    options[first_option][second_option] = third_options

                input_type = 'cascading select'
                options = options
                xpath = child
            
            elif 'issue-link-edit' in child_class :
                input_type = 'issue-link-edit'
                select_item_list = child.find_elements(By.TAG_NAME, 'select')
                options = (list(map(lambda x : x.text, Select(select_item_list[0]).options)), list(map(lambda x : x.get_attribute("innerText").strip(), select_item_list[1].find_elements(By.TAG_NAME, 'option'))))
                xpath = child

            # elif 'aui-field-labelpicker' in child_class :
            #     input_type = 'labelpicker'
            #     options = ""
            #     xpath = child
            elif 'aui-field-labelpicker' in child_class :
                input_type = 'labelpicker'
                options = list(map(lambda x : x.get_attribute("innerText").strip(), child.find_elements(By.TAG_NAME, 'li')))
                xpath = child

            else :
                select_item_list = child.find_elements(By.TAG_NAME, 'select')
                input_list = child.find_elements(By.TAG_NAME, 'input')
                textarea_list = child.find_elements(By.TAG_NAME, 'textarea')

                if len(select_item_list) == 1 :
                    input_type = 'select'
                    options = list(map(lambda x : x.get_attribute("innerText").strip(), select_item_list[0].find_elements(By.TAG_NAME, 'option')))
                    xpath = select_item_list[0]
                elif len(input_list) == 1:
                    input_type = 'input'
                    options = ''
                    xpath = child
                elif len(textarea_list) == 1 :
                    input_type = 'textarea'
                    options = ''' '''
                    xpath = child
                else :
                    input_type = 'Something Went Wrong'
                    options = 'Something Went Wrong'
                    xpath = 'Something Went Wrong'

            print(field_name)


        elif 'fieldset' in child.tag_name :

            # find field name 
            field_name = child.find_element(By.TAG_NAME, 'legend').get_attribute("innerText").split('\n')[0]

            checkbox_elem = child.find_elements(By.TAG_NAME, 'label')
            file_input_elem = child.find_elements(By.CLASS_NAME, 'file-input-list')
            if len(checkbox_elem) > 0 :
                input_type = 'fieldset checkbox' 
                options = list(map(lambda x : x.text, checkbox_elem))
                xpath = child
            
            elif len(file_input_elem) > 0 :
                input_type = 'fieldset file-input-list' 
                options = []
                xpath = child
        
        input_form[field_name] = {
            'type' : input_type,
            'xpath' : xpath,
            'options' : options,
            'required' : required 
        }
        if input_type in ['cascading select', 'issue-link-edit'] :
            input_form[field_name+'_sub'] = {
            'type' : input_type,
            'xpath' : xpath,
            'options' : options,
            'required' : required 
        }

    return input_form

In [64]:
input_form = get_project_input_fields_json(driver, num_recommend_opton=10)

Summary
Security Level
Priority
From
Product Type
Test Method(M)
Problem Type(H)
Problem Type(M)
Operation Type(H)
Operation Type(M)
Due Date
Component/s
Affects Version/s
Fix Version/s
Deployed Server Version/s
Master Fix Version
Master commit URL
Assignee
Multi Assignee
Vehicle Code
Environment
Reproducibility
Description
Original Estimate
Remaining Estimate
Labels
Additional Labels
Feature ID
TeMS ID
Baseline start date
Baseline end date
Progress
Linked Issues
OEM 배포관리 NO
Region(Multi Select)
품질 분류
Fix Version(LGE)
Vehicle_Code_Std
LG하모니 연동


In [118]:
tab = driver.find_element(By.ID, "tab-0")
tab.find_elements(By.CLASS_NAME, "group")

[<selenium.webdriver.remote.webelement.WebElement (session="6531dd233cbceb87074eb291e1c76f9e", element="db2514cb-28ab-435e-a10d-2acf9a81f6d9")>,
 <selenium.webdriver.remote.webelement.WebElement (session="6531dd233cbceb87074eb291e1c76f9e", element="dc400903-03cf-467f-9adb-c0c0f1d17f0e")>]

In [62]:
len(tab.find_elements(By.XPATH, "*"))

58

In [63]:
list(map(lambda x : x.get_attribute('class'), tab.find_elements(By.XPATH, "*")))

['field-group',
 'field-group',
 'field-group',
 'field-group',
 'field-group aui-field-cascadingselect',
 'field-group',
 'field-group',
 'field-group',
 'field-group aui-field-cascadingselect',
 'field-group',
 'field-group aui-field-cascadingselect',
 'field-group aui-field-datepicker',
 'field-group aui-field-componentspicker frother-control-renderer',
 'group',
 'field-group aui-field-versionspicker frother-control-renderer',
 'field-group aui-field-versionspicker frother-control-renderer',
 'field-group aui-field-labelpicker',
 'field-group aui-field-versionspicker frother-control-renderer',
 'field-group',
 'field-group',
 'field-group',
 'field-group aui-field-labelpicker',
 'field-group',
 'field-group',
 'field-group',
 'field-group',
 'field-group',
 'hidden',
 'group',
 'field-group aui-field-labelpicker',
 'field-group aui-field-labelpicker',
 'field-group',
 'field-group',
 'field-group aui-field-datepicker',
 'field-group aui-field-datepicker',
 'field-group aui-field-da

In [12]:
num_recommend_opton = 10


tab = driver.find_element(By.ID, "tab-0")
# field_items = tab.find_elements(By.CLASS_NAME, "field-group") + tab.find_elements(By.TAG_NAME, "fieldset")
# field_selet_items = list(map(lambda x : {x.text.split('\n')[0]: x.find_elements(By.TAG_NAME, 'select')}, field_items))
children = tab.find_elements(By.XPATH, "*")

input_form = {}
field_name_temp = ""
count = 0

for child in children :

    # initialize inputs
    input_type = None
    xpath = None
    options = None
    required = False

    # check input element is required or not
    if  len(child.find_elements(By.CLASS_NAME, 'icon-required')) > 0 :
        required = True
    else :
        required = False

    child_class = child.get_attribute('class')
    if 'field-group' in child_class :

        i = child

        field_name = child.find_element(By.TAG_NAME, 'label').get_attribute("innerText").split('\n')[0]

        if 'aui-field-datepicker' in child_class :
            input_type = 'date'
            options = ""
            xpath = child

        elif 'frother-control-renderer' in child_class :
            input_type = 'rendering select'
            select_item_list = child.find_elements(By.TAG_NAME, 'select')
            option_list = select_item_list[0].find_elements(By.TAG_NAME, 'option')
            if len(option_list) > num_recommend_opton :
                option_list = option_list[:num_recommend_opton]
            options = list(map(lambda x : x.get_attribute("innerText").strip(), option_list))
            xpath = child
        
        elif 'aui-field-cascadingselect' in child_class :
            select_item_list = child.find_elements(By.TAG_NAME, 'select')
            if child.get_attribute('style') != "display: none;" :
                first_select = Select(select_item_list[0])
                first_options = list(map(lambda x : x.text, first_select.options))
                second_select = Select(select_item_list[1])
                options = {}
                for first_option in first_options :
                    first_select.select_by_visible_text(first_option)
                    second_options = list(map(lambda x : x.text, second_select.options))
                    options[first_option] = second_options
            else :                        
                last_added_key = list(input_form.keys())[-1]
                first_select = Select(input_form[last_added_key]['xpath'])
                first_options = input_form[last_added_key]['options']
                second_select = Select(select_item_list[0])
                third_select = Select(select_item_list[1])
                # options = "tbd"
                options = {}
                for first_option in first_options :
                    if first_option :
                        first_select.select_by_visible_text(first_option)
                        second_options = list(map(lambda x : x.text, second_select.options))
                        
                        options[first_option] = {}

                        for second_option in second_options :
                            if second_option :
                                second_select.select_by_visible_text(second_option)
                                third_options = list(map(lambda x : x.text, third_select.options))
                                
                                options[first_option][second_option] = third_options

            input_type = 'cascading select'
            options = options
            xpath = child
        
        elif 'issue-link-edit' in child_class :
            input_type = 'issue-link-edit'
            select_item_list = child.find_elements(By.TAG_NAME, 'select')
            options = (list(map(lambda x : x.text, Select(select_item_list[0]).options)), list(map(lambda x : x.get_attribute("innerText").strip(), select_item_list[1].find_elements(By.TAG_NAME, 'option'))))
            xpath = child

        # elif 'aui-field-labelpicker' in child_class :
        #     input_type = 'labelpicker'
        #     options = ""
        #     xpath = child

        else :
            select_item_list = child.find_elements(By.TAG_NAME, 'select')
            input_list = child.find_elements(By.TAG_NAME, 'input')
            textarea_list = child.find_elements(By.TAG_NAME, 'textarea')

            if len(select_item_list) == 1 :
                input_type = 'select'
                options = list(map(lambda x : x.get_attribute("innerText").strip(), select_item_list[0].find_elements(By.TAG_NAME, 'option')))
                xpath = select_item_list[0]
            elif len(input_list) == 1:
                input_type = 'input'
                options = ''
                xpath = child
            elif len(textarea_list) == 1 :
                input_type = 'textarea'
                options = ''' '''
                xpath = child
            else :
                input_type = 'Something Went Wrong'
                options = 'Something Went Wrong'
                xpath = 'Something Went Wrong'

        print(field_name)


    elif 'fieldset' in child.tag_name :

        # find field name 
        field_name = child.find_element(By.TAG_NAME, 'legend').get_attribute("innerText").split('\n')[0]

        checkbox_elem = child.find_elements(By.TAG_NAME, 'label')
        file_input_elem = child.find_elements(By.CLASS_NAME, 'file-input-list')
        if len(checkbox_elem) > 0 :
            input_type = 'fieldset checkbox' 
            options = list(map(lambda x : x.text, checkbox_elem))
            xpath = child
        
        elif len(file_input_elem) > 0 :
            input_type = 'fieldset file-input-list' 
            options = []
            xpath = child
    
    input_form[field_name] = {
        'type' : input_type,
        'xpath' : xpath,
        'options' : options,
        'required' : required 
    }
    if input_type in ['cascading select', 'issue-link-edit'] :
        input_form[field_name+'_sub'] = {
        'type' : input_type,
        'xpath' : xpath,
        'options' : options,
        'required' : required 
    }


Summary
Security Level
Priority
From
Product Type
Test Method(M)
Problem Type(H)
Problem Type(M)
Operation Type(H)
Operation Type(M)
Due Date
Component/s
Affects Version/s
Fix Version/s
Deployed Server Version/s
Master Fix Version
Master commit URL
Assignee
Multi Assignee
Vehicle Code
Environment
Reproducibility
Description
Original Estimate
Remaining Estimate
Labels
Additional Labels
Feature ID
TeMS ID
Baseline start date
Baseline end date
Progress
Linked Issues
OEM 배포관리 NO
Region(Multi Select)
품질 분류
Fix Version(LGE)
Vehicle_Code_Std
LG하모니 연동


In [14]:
input_form

{'Summary': {'type': 'input',
  'xpath': <selenium.webdriver.remote.webelement.WebElement (session="62d713199d663e39561d88b73932a1f1", element="658c163b-55a4-4247-b9df-4f406418f051")>,
  'options': '',
  'required': True},
 'Security Level': {'type': 'select',
  'xpath': <selenium.webdriver.remote.webelement.WebElement (session="62d713199d663e39561d88b73932a1f1", element="7d1e54ae-837e-419f-ac55-4be01cf672d4")>,
  'options': ['None', 'Private(비공개)', 'Public(공개)'],
  'required': True},
 'Priority': {'type': 'select',
  'xpath': <selenium.webdriver.remote.webelement.WebElement (session="62d713199d663e39561d88b73932a1f1", element="3396320e-1899-4095-85f2-b8d8afe7ee4d")>,
  'options': ['Top', 'A', 'B', 'C'],
  'required': False},
 'From': {'type': 'select',
  'xpath': <selenium.webdriver.remote.webelement.WebElement (session="62d713199d663e39561d88b73932a1f1", element="d8f85f69-5e62-4fcc-8de5-cc29eae7d582")>,
  'options': ['품질(내부)/Internal QA Team',
   'VOC',
   'ISV',
   'HMC',
   'MOBIS'

In [134]:
pip install xlsxwriter

     ------------------------------------ 152.8/152.8 kB 758.2 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: C:\Users\HAEA_MI\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [165]:
import xlsxwriter

workbook = xlsxwriter.Workbook('data_validate.xlsx')
# workbook.add_format({'border' : 5})
worksheet = workbook.add_worksheet()
worksheet.freeze_panes(1, 1)

# column numbering
for i in range(1,100) :
    worksheet.write(0, i, str(i))
    worksheet.set_column("A:ZZ", 30)

worksheet.write_column('A2', list(input_form.keys())+['log', 'attachments', 'upload status'])

for i, key in enumerate(input_form.keys()) :

    # required 
    if input_form[key]['required'] :
        worksheet.set_row(i+1, 15, workbook.add_format({'bold': True, 'bg_color': 'yellow'}))

    # set textarea hight
    if input_form[key]['type'] == 'textarea' :
        worksheet.set_row(i+1, 100)

    # set validation
    if input_form[key]['type'] in ['select', 'rendering select', 'fieldset checkbox', 'aui-field-labelpicker'] :
        
        worksheet_name = key.replace('/', '_').replace(' ', '_').replace('(', '').replace(')', '').lower()
        worksheet_key = workbook.add_worksheet(worksheet_name)
        options = list(map(lambda x : str(x), input_form[key]['options']))
        worksheet_key.write_column('A1', options)
        worksheet.data_validation('B{}:ZZ{}'.format(i+2, i+2), {
            'validate' : 'list',
            'show_error' : False,
            'error_type' : 'information',
            'source': "={}!A1:A{}".format(worksheet_name, len(options))
        })
    elif input_form[key]['type'] in ['cascading select'] :
        
        # parse option table
        option_table = None
        if all(isinstance(n[1], dict) for n in input_form[key]['options'].items()) :
            option_table = list(list(input_form[key]['options'].items())[0][1].items())
        else :
            option_table = list(input_form[key]['options'].items())
        
        if '_sub' in key :
            parent_key = key.split('_sub')[0]
            worksheet_name = parent_key.replace('/', '_').replace(' ', '_').replace('(', '').replace(')', '').lower()
            
            command_head = ''
            command_end = '"not supported option"'
            for table_row_idx, row in enumerate(option_table) :
                title, items = row
                column_alphabet = xlsxwriter.utility.xl_col_to_name(len(items))
                command_head+= 'IF(B{object_row}="{first_option}", {worksheet}!B{row_number}:{column_alphabet}{row_number}, '.format(first_option=title, worksheet=worksheet_name, object_row=i+1, row_number=1+table_row_idx, column_alphabet=column_alphabet)
                command_end+=')'

            command = command_head + command_end

        else :
            worksheet_name = key.replace('/', '_').replace(' ', '_').replace('(', '').replace(')', '').lower()
            worksheet_key = workbook.add_worksheet(worksheet_name)
            options = list(map(lambda x : x[0], option_table))
            
            # fill out the chart
            worksheet_key.write_column('A1', options)
            for idx, elem_tuple in enumerate(option_table) :
                row_elem_list = elem_tuple[1]
                worksheet_key.write_row('B{}'.format(idx+1), row_elem_list)    
                

            command = "={}!A1:A{}".format(worksheet_name, len(options))
        
        # set validation
        worksheet.data_validation('B{}:ZZ{}'.format(i+2, i+2), {
            'validate' : 'list',
            'error_type' : 'information',
            'source': command
        })
    

workbook.close()

In [128]:
testlist = list(filter(lambda x : x[1]['type'] == 'cascading select', list(input_form.items())))

In [75]:
innerlist = list(map(lambda x : x[1], testlist[2][1]['options'].items()))

In [ ]:
input_form['']['options']

In [141]:
xlsxwriter.utility.xl_col_to_name(100)

'CW'

11

In [87]:
innerlist = list(map(lambda x : x[1], testlist[3][1]['options'].items()))
temp = innerlist[0]
if type(temp) == dict :
    for i in range(1, len(innerlist)) :
        if not temp == innerlist[i] and innerlist[i] != {} :
            print('{} {} is not equal'.format(i-1, i))
else :
    print('type of inner list item is not dictionary')

'fieldset'

In [115]:
key = 'Product Type'
key = 'Problem Type(M)'

option_table = None
# option_table = list(n[1].items())
if all(isinstance(n[1], dict) for n in input_form[key]['options'].items()) :
    option_table = list(list(input_form[key]['options'].items())[0][1].items())
    print('dict', option_table)
    
else :
    # option_table = list(list(input_form[key]['options'].items())[1].items())
    option_table = list(input_form[key]['options'].items())
    print('not dict')

dict [('None', ['None']), ('검색/Search', ['None', '초성검색/Initial Letter', '완성어검색/Complete Language', '혼합어검색/Mixed Language', '성/시 검색/State or Provice', '업종명검색/POI Category', '키워드검색/Keyword', '동명칭검색/House Number', '이명명칭검색/Binomial Name', '전화번호검색/Phone Number', '주소검색/Address', '새주소검색/New Address', '도로명검색/Road Name', '중간어검색/Middle Letter', '테마검색/Theme', '상/하위POI검색/Subordinate POI', '멀티POI검색/Multi POI', '유가검색/Oil Price', '정렬/Alignment', '최근검색/Recent', '주변검색/Near Position', '등록지 검색/Bookmark', '지하철역검색/Subway', '좌표검색/Coordinate', 'AND검색(명칭+명칭, 지역+명칭)/Combination', '프리미엄우편번호검색/Postal Code', '교차로검색/Intersection', '아파트검색/Apartment', 'Emergency', '필기인식/Writing Recognition', '자동완성/Auto Complete', 'Presearch Count', 'Quick Speller', 'Freeway 입(Enterance)/출구(Exit) 검색']), ('탐색/Route Calculation', ['None', '장거리 탐색/Long Distance', '중거리 탐색/Middle Distance', '단거리 탐색/Short Distance', '추천 탐색/Recommandation', '고속 탐색/Highway', '무료 탐색/Freeway', '거리 탐색/Distance', '교통 탐색/Traffic', '고속도로 최소화/Minimize Highway', 'Fe

In [112]:
list(input_form[key]['options'].items())

[('None', ['None']),
 ('HW', ['None', 'Navi', 'Black Box']),
 ('SW', ['None', 'Navi']),
 ('DB', ['None', '해당없음/Not Applicable']),
 ('SD', ['None', '해당없음/Not Applicable'])]

In [108]:
n = list(input_form[key]['options'].items())[0]
list(n[1].items())

[('None', ['None']),
 ('검색/Search',
  ['None',
   '초성검색/Initial Letter',
   '완성어검색/Complete Language',
   '혼합어검색/Mixed Language',
   '성/시 검색/State or Provice',
   '업종명검색/POI Category',
   '키워드검색/Keyword',
   '동명칭검색/House Number',
   '이명명칭검색/Binomial Name',
   '전화번호검색/Phone Number',
   '주소검색/Address',
   '새주소검색/New Address',
   '도로명검색/Road Name',
   '중간어검색/Middle Letter',
   '테마검색/Theme',
   '상/하위POI검색/Subordinate POI',
   '멀티POI검색/Multi POI',
   '유가검색/Oil Price',
   '정렬/Alignment',
   '최근검색/Recent',
   '주변검색/Near Position',
   '등록지 검색/Bookmark',
   '지하철역검색/Subway',
   '좌표검색/Coordinate',
   'AND검색(명칭+명칭, 지역+명칭)/Combination',
   '프리미엄우편번호검색/Postal Code',
   '교차로검색/Intersection',
   '아파트검색/Apartment',
   'Emergency',
   '필기인식/Writing Recognition',
   '자동완성/Auto Complete',
   'Presearch Count',
   'Quick Speller',
   'Freeway 입(Enterance)/출구(Exit) 검색']),
 ('탐색/Route Calculation',
  ['None',
   '장거리 탐색/Long Distance',
   '중거리 탐색/Middle Distance',
   '단거리 탐색/Short Distance',
   '추천 탐색/Recomm

In [ ]:
        # select_item_list = i.find_elements(By.TAG_NAME, 'select')          
        # match len(select_item_list) :
        #     case 0 :
        #         # non-select element
        #         input_list = i.find_elements(By.TAG_NAME, 'input')
        #         if len(input_list) == 1 :
        #             input_type = "text"
        #             xpath = input_list[0]
        #         else :
        #             textarea_list = i.find_elements(By.TAG_NAME, 'textarea')
        #             if len(textarea_list) == 1 :
        #                 input_type = "textarea"
        #                 xpath = textarea_list[0]
        #         options = None
        #     case 1 :
        #         # general select element
        #         input_type = 'select'
        #         option_list = select_item_list[0].find_elements(By.TAG_NAME, 'option')
        #         if len(option_list) > num_recommend_opton :
        #             option_list = option_list[:num_recommend_opton]
        #         options = list(map(lambda x : x.get_attribute("innerText").strip(), option_list))

        #         xpath = select_item_list[0]
        #     case 2 :
        #         # nested select elemnt
        #         input_type = 'variant select'

        #         if 'aui-field-cascadingselect' in i.get_attribute('class') :
        #             if count != 0 :         
        #                 input_type = 'three cascading select'
                        
        #                 first_select = Select(input_form[field_name_temp]['xpath'])
        #                 first_options = input_form[field_name_temp]['options']
        #                 second_select = Select(select_item_list[0])
        #                 third_select = Select(select_item_list[1])

        #                 # options = "tbd"
        #                 options = {}
        #                 for first_option in first_options :
        #                     if first_option :
        #                         first_select.select_by_visible_text(first_option)
        #                         second_options = list(map(lambda x : x.text, second_select.options))
                                
        #                         options[first_option] = {}

        #                         for second_option in second_options :
        #                             if second_option :
        #                                 second_select.select_by_visible_text(second_option)
        #                                 third_options = list(map(lambda x : x.text, third_select.options))
                                        
        #                                 options[first_option][second_option] = third_options
        #             else :
        #                 input_type = 'two cascading select'
        #                 first_select = Select(select_item_list[0])
        #                 # first_options = input_form[field_name_temp]['options']
        #                 first_options = list(map(lambda x : x.text, first_select.options))
        #                 second_select = Select(select_item_list[1])

        #                 options = {}
        #                 for first_option in first_options :
        #                     first_select.select_by_visible_text(first_option)
        #                     second_options = list(map(lambda x : x.text, second_select.options))
                            
        #                     options[first_option] = second_options                    
                    
        #         else :
        #             input_type = 'variant select'
        #             options = [list(map(lambda x : x.text, Select(select_item_list[0]).options)), list(map(lambda x : x.text, Select(select_item_list[1]).options))]
        #         xpath = i            
        #     case _ :
        #         input_type = 'Error'
        #         options = "Error"
                # xpath = None
